In [246]:
from pyrosetta import *
from pyrosetta.rosetta import *
from MiningMinima import *

# Developing subroutines to integrate over rigid body dofs

## Initialize stuff

In [263]:
seq1, seq2 = 'u',  'a'
pose = protocols.recces.pose_setup_turner(seq1, seq2)
n_res = len(seq1) + len(seq2)

# set fold tree using usual chi torsion atoms
ft = FoldTree(n_res)
ft.new_jump(1, n_res, n_res/2)
ft.set_jump_atoms(1, pose.residue(1).atom_name(pose.residue(1).chi_atoms(1)[4]),
    pose.residue(n_res).atom_name(pose.residue(n_res).chi_atoms(1)[4]))
pose.fold_tree(ft)

# set jump dofs in movemap
movemap = MoveMap()
movemap.set_jump(1, True)
active_jump = pose.jump(1)

# get scorefxn
scorefxn = core.scoring.ScoreFunctionFactory.create_score_function('stepwise/rna/turner')

In [264]:
pmm = PyMOLMover()
pmm.apply(pose)

## Set up multifunc and minimize

In [265]:
def array_to_vector1(dofs):
    vector_of_dofs = Vector1(list(dofs))
    return vector_of_dofs

In [266]:
# taken from MiningMinima class
def set_up_multifunc(pose, movemap, scorefxn):
    # set up minimizer map
    min_map = core.optimization.MinimizerMap()
    min_map.setup(pose, movemap)

    start_score = scorefxn(pose)
    pose.energies().set_use_nblist(pose, min_map.domain_map(), True)
    multifunc = core.optimization.AtomTreeMultifunc(pose, min_map, scorefxn)
    scorefxn.setup_for_minimizing(pose, min_map)
    scorefxn.setup_for_derivatives(pose)

    min_map = min_map
    multifunc = multifunc
    
    return multifunc, min_map

In [267]:
multifunc, min_map = set_up_multifunc(pose, movemap, scorefxn)
min_dofs = Vector1([0.0]*min_map.nangles())
min_map.copy_dofs_from_pose(pose,min_dofs)
start_func_val = multifunc(min_dofs)
theta = Vector1(list(min_dofs))
dE_dtheta = Vector1(list(min_dofs))
multifunc.dfunc(theta,dE_dtheta)

In [268]:
min_options = core.optimization.MinimizerOptions(
    'lbfgs_armijo_nonmonotone', 1e-15, True, False, False)
min_options.nblist_auto_update(True)
minimizer = core.optimization.Minimizer(multifunc, min_options)

In [269]:
min_dofs = Vector1(list(theta))
minimizer.run(min_dofs)
min_map.copy_dofs_to_pose(pose, min_dofs)
pmm.apply(pose)

In [270]:
pmm.keep_history(True)
min_dofs = np.array(min_dofs)
new_dofs = np.array(min_dofs)

In [271]:
hessian = hessian_at_min(min_dofs, multifunc)

In [272]:
from numpy.linalg import eigh

In [273]:
[eigenvalues, modes] = eigh(hessian)

In [274]:
print active_jump.get_translation()

      6.234588052111575     0.08855427053947298    0.006925434051985979


In [275]:
min_map.reset_jump_rb_deltas(pose, array_to_vector1(new_dofs))

In [276]:
total_partition, total_harmonic, mode_scans = compute_total_partition(min_dofs, multifunc, modes, eigenvalues, limit=np.pi)

In [277]:
print multifunc(array_to_vector1(min_dofs)) + 0.5*np.log(np.linalg.det(hessian)) - 3*np.log(2*np.pi)

1.98059933864


## Expression for the dissociation constant in terms of molecular partition function 

The dissociation constant is given by 

$$ K_{\text{d}} = \frac{P_{\text{unbound}}}{P_{\text{bound}}} = \frac{8\pi^2V}{\int_V e^{-\beta E} dr}$$

Denote the integral in the denominator as $Z$. 
And since $\Delta G_{\text{init}}$ is given by $kT \ln K_{\text{d}}$:

$$\Delta G_{\text{init}} = -kT \ln Z + kT \ln 8\pi ^2V $$

N.B. The volume is defined in terms of the reference concentration: $V = \frac{1}{C°} \rightarrow kT \ln 8\pi^2V=kT\ln 8\pi^2 -kT\ln{\text{6.022e-4}}$

In [278]:
multifunc(array_to_vector1(min_dofs)) - total_partition + np.log(8*np.pi**2) - np.log(6.022e-4)

7.7382854563610106